In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Import Libraries

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

import string
from heapq import nlargest
import sys
import spacy
!pip install truecase
import truecase
from spacy import displacy
from collections import Counter
from nltk.tag import StanfordNERTagger
import en_core_web_sm
nlp = en_core_web_sm.load()
from pprint import pprint
!pip install unidecode
from unidecode import unidecode
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
import warnings
from tqdm._tqdm_notebook import tqdm_notebook
from scipy.spatial import distance
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.cluster.util import cosine_distance
import tensorflow as tf
import transformers
!pip install sent2vec
from sent2vec.vectorizer import Vectorizer as S2vectorizer
tqdm_notebook.pandas()
warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
!pip install pickle5
import pickle5 as pickle  
import networkx as nx

#import pipelines
from transformers import pipeline
nlp2 = pipeline("question-answering")

pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Preprocessing

In [ ]:
stop_words = stopwords.words("english")

def clean_normalcase_stop_lem(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    text = ' '.join([wordnet_lemmatizer.lemmatize(i) for i in text.split()])
    return text

def clean_normalcase_nostop_lem(text):
    text = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text

def clean_lowercase_stop_lem(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    text = ' '.join([wordnet_lemmatizer.lemmatize(i) for i in text.split()])
    return text.lower()

def clean_lowercase_nostop_lem(text):
    text = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
#     text = ' '.join([ps.stem(word) for word in text.split() if word not in stop_words])
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text.lower()

In [ ]:
def extractive_summariser(text):

  # If the length of the text is greater than 20, take a 10th of the sentences

  length = 3

  # Remove punctuation
  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = ''.join(nopunc)
  # Remove stopwords
  processed_text =[word for word in nopunc.split() if word.lower() not in nltk.corpus.stopwords.words('english')]

  # Create a dictionary to store word frequency
  word_freq = {}
  # Enter each word and its number of occurrences
  for word in processed_text:
    if word not in word_freq:
      word_freq[word] = 1
    else:
      word_freq[word] = word_freq[word] + 1

  # Divide all frequencies by max frequency to give store of (0, 1]
  max_freq = max(word_freq.values())
  for word in word_freq.keys():
    word_freq[word] = (word_freq[word]/max_freq)

  # Create a list of the sentences in the text
  sent_list = nltk.sent_tokenize(text)
  # Create an empty dictionary to store sentence scores
  sent_score = {}
  for sent in sent_list:
    for word in nltk.word_tokenize(sent.lower()):
      if word in word_freq.keys():
        if sent not in sent_score.keys():
          sent_score[sent] = word_freq[word]
        else:
          sent_score[sent] = sent_score[sent] + word_freq[word]

  summary_sents = nlargest(length, sent_score, key = sent_score.get)
  summary = ' '.join(summary_sents)
  return summary

In [ ]:
ldf['cleaned_normalcase_stop_lem'] = df['context'].progress_apply(clean_normalcase_stop_lem)
df['cleaned_normalcase_nostop_lem'] = df['context'].progress_apply(clean_normalcase_nostop_lem)
df['cleaned_lowercase_stop_lem'] = df['context'].progress_apply(clean_lowercase_stop_lem)
df['cleaned_lowercase_nostop_lem'] = df['context'].progress_apply(clean_lowercase_nostop_lem)
df['ner_combined'] = df['context_ner'].apply(lambda x: ' '.join(x).strip('[').strip(']').replace(',',''))


In [ ]:
df['extractive_summarized_3_sent'] = df['context'].progress_apply(extractive_summariser)
df['extractive_summarized_3_sent'] = df['extractive_summarized_3_sent'].progress_apply(clean_lowercase_nostop_lem)

In [ ]:
df.to_pickle('/content/gdrive/MyDrive/tmproj/2_apr_brian.pkl')

## Load Data

In [ ]:
#Read the pickle file
with open('/content/gdrive/MyDrive/tmproj/2_apr_brian.pkl', "rb") as fh:
  df = pickle.load(fh)

In [ ]:
df.head(3)

,context,context_ner,cleaned_normalcase_stop_lem,cleaned_normalcase_nostop_lem,cleaned_lowercase_stop_lem,cleaned_lowercase_nostop_lem,ner_combined,extractive_summarized_3_sent
0,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".","{the late 1990s, american, grammy awards, knowles-carter, 2003, mathew knowles, crazy in love, september 4, 1981, love, destiny's child, one, r&b, beyonce, texas, baby boy, houston, 100}",Beyonce Giselle Knowles Carter bi:'janseI bee YON say born September 4 1981 is an American singer songwriter record producer and actress Born and raised in Houston Texas she performed in various singing and dancing competitions as a child and rose to fame in the late 1990s as lead singer of R B girl group Destiny s Child Managed by her father Mathew Knowles the group became one of the world s best selling girl groups of all time Their hiatus saw the release of Beyonce s debut album Dangerously in Love 2003 which established her as a solo artist worldwide earned five Grammy Awards and featured the Billboard Hot 100 number one singles Crazy in Love and Baby Boy,Beyonce Giselle Knowles Carter bi:'janseI bee YON say born September 4 1981 American singer songwriter record producer actress Born raised Houston Texas performed various singing dancing competitions child rose fame late 1990s lead singer R B girl group Destiny s Child Managed father Mathew Knowles group became one world s best selling girl groups time Their hiatus saw release Beyonce s debut album Dangerously Love 2003 established solo artist worldwide earned five Grammy Awards featured Billboard Hot 100 number one singles Crazy Love Baby Boy,beyonce giselle knowles carter bi:'jansei bee yon say born september 4 1981 is an american singer songwriter record producer and actress born and raised in houston texas she performed in various singing and dancing competitions as a child and rose to fame in the late 1990s as lead singer of r b girl group destiny s child managed by her father mathew knowles the group became one of the world s best selling girl groups of all time their hiatus saw the release of beyonce s debut album dangerously in love 2003 which established her as a solo artist worldwide earned five grammy awards and featured the billboard hot 100 number one singles crazy in love and baby boy,beyonce giselle knowles carter bi:'jansei bee yon say born september 4 1981 american singer songwriter record producer actress born raised houston texas performed various singing dancing competition child rose fame late 1990s lead singer r b girl group destiny s child managed father mathew knowles group became one world s best selling girl group time their hiatus saw release beyonce s debut album dangerously love 2003 established solo artist worldwide earned five grammy awards featured billboard hot 100 number one single crazy love baby boy,american 2003 grammy awards texas september 4 1981 baby boy mathew knowles beyonce love 100 r&b the late 1990s knowles-carter one houston destiny's child crazy in love,born raised houston texas performed various singing dancing competition child rose fame late 1990s lead singer r b girl group destiny child hiatus saw release beyonce debut album dangerously love 2003 established solo artist worldwide earned five grammy award featured billboard hot 100 number one single crazy love baby boy beyon

## NER Generation

In [ ]:
pprint([(X.text, X.label_) for X in nlp("America is a country").ents])

[('America', 'GPE')]


In [ ]:
def generate_ners(text):
    result = set()
    for X in nlp(text).ents:
        result.add(unidecode(X.text.lower()))
    return result

In [ ]:
df = pd.read_csv('../data/SQuAD_csv.csv', encoding='utf-8').loc[:, ['context']]
df = df.drop_duplicates(subset=['context'])
df['context_ner'] = df['context'].progress_apply(generate_ners)
# df['context'] = df['context'].apply(lambda x: str(x).lower())

## Method 1: NER TF-IDF

In [ ]:
# TF-IDF Vectorizer for ner_combined column
ner_vectorizer = TfidfVectorizer(ngram_range=(1,2))
ner_tfidf = ner_vectorizer.fit_transform(df['ner_combined'])

In [ ]:
# Query preprocessing for NER document retrieval
def query_to_ner_str(query):
    result = re.sub('[%s]' % re.escape(string.punctuation), '', query)
    result = re.sub('\s{2,}', " ", result)
    result = ' '.join([word for word in result.split() if word.lower() not in stop_words])
    result = ' '.join([X.text.lower() for X in nlp(truecase.get_true_case(result)).ents])

    if result == '':
        return None
    return result

# Testing query_to_ner_str function
query_to_ner_str('what is the political system of the islamic republic based on')

'islamic republic'

In [ ]:
def document_retrieval_ner(df, ner_vectorizer, ner_tfidf, query):
    ner_str = query_to_ner_str(query)

    if ner_str is None:
        return None
    
    print(f"NER String: {ner_str}\n")
    
    ner_matching_ids = get_similar_docs(df, ner_vectorizer, ner_tfidf, ner_str)
    
    return ner_matching_ids

In [ ]:
def get_similar_docs(df, tfidfvectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query

    return: doc with highest tf-idf cosine similarity
    """
    query_tfidf = tfidfvectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    max_sim = max(cosineSimilarities)
    
    if max_sim < 0.05: # not sure whether to set this threshold as some correct answers are like 0.1 similarity
        print("No Matches")
        return None
    else:
        threshold = 0.6 * max_sim
    
    top_doc_ids = set()
    for idx, val in enumerate(cosineSimilarities):
        if val >= threshold:
            top_doc_ids.add((idx,val))
            
    top_doc_ids = sorted(top_doc_ids, key=lambda x: x[1], reverse=True)
            
    print(f"Top Docs: {top_doc_ids}\n")
            
    return top_doc_ids

In [ ]:
query = 'what does polytechnic mean in singapore?'
retrieved_doc_ids = document_retrieval_ner(df, ner_vectorizer, ner_tfidf, query)
print(df.iloc[[i[0] for i in retrieved_doc_ids], df.columns.get_loc('context')].apply(lambda x: [i.strip() for i in x.split('.') if len(i)>1]))

NER String: singapore

Top Docs: [(1383, 0.3412023014328927), (1382, 0.23353586294956077), (8912, 0.20957190592726097)]

7599     [Polytechnics offer three-year diploma courses in fields such as information technology, engineering subjects and other vocational fields, like psychology and nursing, There are 5 polytechnics in Singapore, They are namely:]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

## Method 2: Paragraph TF-IDF

In [ ]:
# TF-IDF Vectorizer for cleaned_lowercase_nostop_lem column
para_vectorizer = TfidfVectorizer(ngram_range=(1,2))
para_tfidf = para_vectorizer.fit_transform(df['cleaned_lowercase_nostop_lem'])

In [ ]:
# Query preprocessing for paragraph document retrieval
def query_normal_cleaning(query):
    result = re.sub('[%s]' % re.escape(string.punctuation), '', query)
    result = re.sub('\s{2,}', " ", result).lower()
    result = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in result.split() if word not in stop_words])

    if result == '':
        return None
    return result

# Testing query_to_ner_str function
query_normal_cleaning('what is the political system of the islamic republic based on')

'political system islamic republic based'

In [ ]:
def document_retrieval_para(df, para_vectorizer, para_tfidf, query):
    cleaned = query_normal_cleaning(query)

    if cleaned is None:
        return None
    
    print(f"Cleaned Query: {cleaned}\n")
    
    para_matching_ids = get_similar_docs(df, para_vectorizer, para_tfidf, cleaned)
    
    return para_matching_ids

In [ ]:
query = 'what does polytechnic mean in singapore?'
retrieved_doc_ids = document_retrieval_para(df, para_vectorizer, para_tfidf, query)
if retrieved_doc_ids:
    print(df.iloc[[i[0] for i in retrieved_doc_ids], df.columns.get_loc('context')].apply(lambda x: [i.strip() for i in x.split('.') if len(i)>1]))

Cleaned Query: polytechnic mean singapore

Top Docs: [(1341, 0.2742364173820103), (1382, 0.25939776890594335), (1360, 0.2366924919912639), (1375, 0.23299376347972817), (1391, 0.2048776630550047), (8918, 0.1885104228610306), (1383, 0.17557414483448747)]

7520     [In Croatia there are many polytechnic institutes and colleges that offer a polytechnic education, The law about polytechnic education in Croatia was passed in 1997]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

## Method 3: NER + Paragraph TF-IDF

In [ ]:
def document_retrieval_ner_para(df, ner_vectorizer, ner_tfidf, para_vectorizer, para_tfidf, query):
    df = df.reset_index()
    ner_str = query_to_ner_str(query)
    ner_docs = []
    
    cleaned = query_normal_cleaning(query)

    if cleaned is None:
        return None

    if ner_str:    
        print('========== EXECUTING NER TF-IDF ==========')
        print(f"NER String: {ner_str}\n")

        ner_matching_ids = get_similar_docs(df, ner_vectorizer, ner_tfidf, ner_str)

        if ner_matching_ids:
          narrowed_paras_df = df.iloc[[i[0] for i in ner_matching_ids],:]
          para_index = narrowed_paras_df.index
                
          # Re-run para_vectorizer and para_tfidf on narrowed down documents
          para_vectorizer = TfidfVectorizer(ngram_range=(1,2))
          para_tfidf = para_vectorizer.fit_transform(narrowed_paras_df['cleaned_lowercase_nostop_lem'])
        else:
          para_index = df.index
    else:
        para_index = df.index
        
    print('=== EXECUTING PARA TF-IDF ===')
    print(f"Query String: {cleaned}\n")
    query_tfidf = para_vectorizer.transform([cleaned])
    cosineSimilarities = cosine_similarity(query_tfidf, para_tfidf).flatten()
    max_sim = max(cosineSimilarities)
    
    if max_sim < 0.05: # not sure whether to set this threshold as some correct answers are like 0.1 similarity
        print("No Matches")
        return None
    else:
        threshold = 0.4 * max_sim
    
    top_doc_ids = set()
    for idx, val in enumerate(cosineSimilarities):
        if val >= threshold:
            top_doc_ids.add((para_index[idx],val))
            
    top_doc_ids = sorted(top_doc_ids, key=lambda x: x[1], reverse=True)
            
    print(f"Top Docs: {top_doc_ids}\n")
            
    return top_doc_ids

In [ ]:
query = input()
print('------------------------------------------------------------------------------')
retrieved_doc_ids = document_retrieval_ner_para(df, ner_vectorizer, ner_tfidf, para_vectorizer, para_tfidf, query)
if retrieved_doc_ids:
    print(df.iloc[[i[0] for i in retrieved_doc_ids], df.columns.get_loc('context')].apply(lambda x: [i.strip() for i in x.split('.') if len(i)>1]))

what does polytechnic mean in singapore
------------------------------------------------------------------------------
========== EXECUTING NER TF-IDF ==========
NER String: singapore

Top Docs: [(1383, 0.3412023014328927), (1382, 0.23353586294956077), (8912, 0.20957190592726097)]

=== EXECUTING PARA TF-IDF ===
Query String: polytechnic mean singapore

Top Docs: [(1382, 0.22061356115579817), (1383, 0.155714613958824)]

7598    [Polytechnics in Singapore provides industry oriented education equivalent to a junior college or sixth form college in the UK, Singapore retains a system similar but not the same as in the United Kingdom from 1970–1992, distinguishing between polytechnics and universities, Unlike the British Polytechnic (United Kingdom) system Singapore Polytechnics do not offer bachelors, masters or PhD degrees, Under this system, most Singaporean students sit for their O-Level examinations after a four or five years of education in secondary school, and apply for a place at ei

## Method 4: Latent Semantic Indexing (LSI)

In [ ]:
from numpy.linalg import svd, norm
from nltk.stem.snowball import EnglishStemmer
from collections import defaultdict, Counter

In [ ]:
class LSI:
    def __repr__(self):
        return 'LSI( terms:{}, documents:{}, index_ready:{})'.format(self.index.__len__(),
                                                                     self.documents.__len__(),
                                                                     not(self.__update_index))
    
    def __init__(self, tokenizer=nltk.word_tokenize,
                 stemmer=EnglishStemmer,
                 stopwords=nltk.corpus.stopwords.words('english'),
                 variance=0.9):
        '''
        >>> queries cannot work unless __update_index is false.        
        '''
        self.stemmer = stemmer()
        self.tokenizer = tokenizer
        self.stopwords = stopwords
        
        self.documents = {}
        self.index = defaultdict(Counter)
        
        self.A = None # term document matrix
        self.U = None # output of svd
        self.S = None # output of svd
        self.V = None # output of svd
        
        self.term_rep = np.array(None) # reduced representation of terms after svd
        self.doc_rep = np.array(None)  # reduced representation of documents after svd
        
        self.__update_index = True
        self._term_index_in_A = {}
        self._document_index_in_A = {}
        
        self.k = None # reduced dimension after SVD
        self.variance = variance # variance to retain after SVD
            
        
    def add_doc(self, document, document_id):
        '''
        add terms into vocabulary.
        add document 
        '''
        if document_id in self.documents:
            print('document_id : {} already indexed.'.format(document_id))
            return False
        
        for token in [t.lower() for t in self.tokenizer(document) if t.isalpha()]:
            if token in self.stopwords:
                continue;
            if self.stemmer:
                token = self.stemmer.stem(token)
                
            # add this token to defaultdict(Counter)
            # this document's count is increased by 1 for this token's Counter
            self.index[token].update({document_id:1})
        
        self.__update_index = True # update flag to rebuild index
        self.documents[document_id] = document # add document to documents
        return True
    
    
    def _svd_A(self):
        '''
        Perform SVD on A and update the U,S,V matrices
        '''
        self.U, self.S, self.V = svd(self.A)
        
    
    def _get_k_for_svd(self):
        '''
        Finds the value for k after SVD such that specified variance is retained
        returns k : int
        '''
        if (self.S is not None):
            sum = 0
            k = 0
            while(sum < self.variance):
                k -=- 1
                sum = self.S[:k].sum() / self.S.sum()
            self.k = k
            return True
        else:
            print('S is not populated.')
            return False

    def rebuild_index(self):
        '''
        >>> set _update_index to false when index is built
        '''
        terms = list(self.index.keys())
        documents = list(self.documents.keys())
        self.A = np.zeros((terms.__len__(), documents.__len__()), dtype='int8')

        self._document_index_in_A = {doc:ix for ix,doc in enumerate(documents)}
        self._term_index_in_A = {term:ix for ix,term in enumerate(terms)}
        
        for term in terms:
            counter = self.index[term]
            term_ix = self._term_index_in_A[term]
            doc_ids = list(self.index[term].keys())
            doc_vals = [counter[x] for x in doc_ids]
            doc_ixs = [self._document_index_in_A[x] for x in doc_ids]
            for ix,doc_id in enumerate(doc_ixs):
                self.A[term_ix][doc_id] = doc_vals[ix]
        print('Term-Document frequency matrix is ready.')
        print('Proceeding to do SVD on the matrix.')
        
        self._svd_A()
        self._get_k_for_svd()
        
        self.doc_rep = self.V[:self.k,:]
        self.term_rep = self.U[:,:self.k]

        print('Index Rebuilt. Setting __update_index to False. Queries can now be performed.')
        self.__update_index = False
        
    def _calc_query_doc_affinity_score(self, query_vector):
        '''
        calculates the query - document affinity score
        '''
        try:
            one_by_query_vector_norm_ = (1/norm(query_vector))
        except ZeroDivisionError:
            one_by_query_vector_norm_ = (1/1e-4)
        affinity_scores = (np.dot(query_vector,self.doc_rep) / norm(self.doc_rep, axis=0)) * one_by_query_vector_norm_
        return affinity_scores
    
    def query(self, query_string, top=5):
        
        if(self.__update_index == True):
            print('Index is not updated. Use rebuild_index()')
            return False
        
        query_vector = []
        for token in [t.lower() for t in self.tokenizer(query_string) if t.isalpha()]:
            if token in self.stopwords:
                continue;
            if self.stemmer:
                token = self.stemmer.stem(token)
            try:
                query_vector.append(self.term_rep[self._term_index_in_A[token], :])
            except KeyError:
                query_vector.append(np.array([0.0] * self.k))
        
        query_vector_mean = np.array(query_vector).mean(axis=0)
        affinity_scores = self._calc_query_doc_affinity_score(query_vector_mean)
        
        res_doc_index = (-affinity_scores).argsort()[:top]
        results = []
        for index in res_doc_index:
            res_doc_id = list(self._document_index_in_A.keys())[index]
            results.append(self.documents[res_doc_id])
            
        return results

In [ ]:
lsi = LSI()

for index, row in df.iterrows():
    lsi.add_doc(row['extractive_summarized_3_sent'], index)

In [ ]:
# runs forever, not sure if viable
lsi.rebuild_index()

Term-Document frequency matrix is ready.
Proceeding to do SVD on the matrix.


## Method 5: Sentence Embedding (Doc2Vec)

In [69]:
doc2vec_df = df.reset_index()

In [72]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = []
for idx, d in doc2vec_df.iterrows():
  print(f"Index: {idx}")
  tagged_data.append(TaggedDocument(d['extractive_summarized_3_sent'].split(), [idx]))
  print(TaggedDocument(d['extractive_summarized_3_sent'].split(), [idx]))
  print('--------------')

tagged_data

Streaming output truncated to the last 5000 lines.
TaggedDocument(['many', 'tried', 'escape', 'incorporation', 'join', 'french', 'resistance', 'desert', 'wehrmacht', 'many', 'running', 'risk', 'family', 'sent', 'work', 'concentration', 'camp', 'german', 'nowadays', 'history', 'recognizes', 'suffering', 'people', 'museum', 'public', 'discussion', 'memorial', 'built', 'commemorate', 'terrible', 'period', 'history', 'part', 'eastern', 'france', 'alsace', 'moselle', 'many', 'men', 'especially', 'answer', 'call', 'immediately', 'pressured', 'volunteer', 'service', 's', 'often', 'direct', 'threat', 'family'], [12357])
--------------
Index: 12358
TaggedDocument(['session', 'take', 'place', 'immeuble', 'louise', 'wei', 'inaugurated', '1999', 'house', 'largest', 'parliamentary', 'assembly', 'room', 'europe', 'democratic', 'institution', 'world', 'ep', 'session', 'take', 'place', 'main', 'council', 'europe', 'building', 'palace', 'europe', 'whose', 'unusual', 'inner', 'architecture', 'become', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
TaggedDocument(['long', 'life', 'fast', 'switching', 'time', 'ability', 'seen', 'broad', 'daylight', 'due', 'high', 'output', 'focus', 'led', 'used', 'brake', 'light', 'car', 'high', 'mounted', 'brake', 'light', 'truck', 'bus', 'turn', 'signal', 'time', 'many', 'vehicle', 'use', 'led', 'rear', 'light', 'cluster', 'use', 'brake', 'improves', 'safety', 'due', 'great', 'reduction', 'time', 'needed', 'light', 'fully', 'faster', 'rise', 'time', '0', '5', 'second', 'faster', 'citation', 'needed', 'incandescent', 'bulb', 'dual', 'intensity', 'circuit', 'rear', 'marker', 'brake', 'led', 'pulsed', 'fast', 'enough', 'frequency', 'create', 'phantom', 'array', 'ghost', 'image', 'led', 'appear', 'eye', 'quickly', 'scan', 'across', 'array'], [17210])
--------------
Index: 17211
TaggedDocument(['light', 'emitting', 'diode', 'well', 'semiconductor', 'laser', 'used', 'send', 'data', 'many', 'type', 'fiber', 'optic', 'cable', 'digital', 'audio', 'toslin

[TaggedDocument(words=['born', 'raised', 'houston', 'texas', 'performed', 'various', 'singing', 'dancing', 'competition', 'child', 'rose', 'fame', 'late', '1990s', 'lead', 'singer', 'r', 'b', 'girl', 'group', 'destiny', 'child', 'hiatus', 'saw', 'release', 'beyonce', 'debut', 'album', 'dangerously', 'love', '2003', 'established', 'solo', 'artist', 'worldwide', 'earned', 'five', 'grammy', 'award', 'featured', 'billboard', 'hot', '100', 'number', 'one', 'single', 'crazy', 'love', 'baby', 'boy', 'beyonce', 'giselle', 'knowles', 'carter', 'bi', 'jansei', 'bee', 'yon', 'say', 'born', 'september', '4', '1981', 'american', 'singer', 'songwriter', 'record', 'producer', 'actress'], tags=[0]),
 TaggedDocument(words=['beyonce', 'took', 'hiatus', 'music', '2010', 'took', 'management', 'career', 'fourth', 'album', '4', '2011', 'subsequently', 'mellower', 'tone', 'exploring', '1970s', 'funk', '1980s', 'pop', '1990s', 'soul', 'marriage', 'rapper', 'jay', 'z', 'portrayal', 'etta', 'james', 'cadillac',

In [73]:
## Train doc2vec model
model = Doc2Vec(tagged_data, vector_size = 20, min_count = 1, epochs = 10)

'''
vector_size = Dimensionality of the feature vectors.
window = The maximum distance between the current and predicted word within a sentence.
min_count = Ignores all words with total frequency lower than this.
alpha = The initial learning rate.
'''

## Print model vocabulary
model.wv.vocab

{'born': <gensim.models.keyedvectors.Vocab at 0x7fcf71a6fe50>,
 'raised': <gensim.models.keyedvectors.Vocab at 0x7fcf71b44490>,
 'houston': <gensim.models.keyedvectors.Vocab at 0x7fcf76386490>,
 'texas': <gensim.models.keyedvectors.Vocab at 0x7fcf71d10450>,
 'performed': <gensim.models.keyedvectors.Vocab at 0x7fcf71a02d90>,
 'various': <gensim.models.keyedvectors.Vocab at 0x7fcf7623d450>,
 'singing': <gensim.models.keyedvectors.Vocab at 0x7fcf76240450>,
 'dancing': <gensim.models.keyedvectors.Vocab at 0x7fcf76240d50>,
 'competition': <gensim.models.keyedvectors.Vocab at 0x7fcf71a6b4d0>,
 'child': <gensim.models.keyedvectors.Vocab at 0x7fcf76351b50>,
 'rose': <gensim.models.keyedvectors.Vocab at 0x7fcf7621e8d0>,
 'fame': <gensim.models.keyedvectors.Vocab at 0x7fcf7621eb10>,
 'late': <gensim.models.keyedvectors.Vocab at 0x7fcf7c0b44d0>,
 '1990s': <gensim.models.keyedvectors.Vocab at 0x7fcf7c0b4dd0>,
 'lead': <gensim.models.keyedvectors.Vocab at 0x7fcf7c0b4710>,
 'singer': <gensim.models.

In [76]:
test_doc = word_tokenize("What does polytechnic mean in Singapore?".lower())
test_doc_vector = model.infer_vector(test_doc)
doc2vec_mostsimilar = model.docvecs.most_similar([test_doc_vector])

for i in doc2vec_mostsimilar:
  print(f"Index / Score: {i}")
  print(f"Context: {doc2vec_df.iloc[i[0],list(doc2vec_df.columns).index('context')]}")

Index / Score: (6605, 0.8398773670196533)
Context: St James Junior School in London, England, offers Sanskrit as part of the curriculum. In the United States, since September 2009, high school students have been able to receive credits as Independent Study or toward Foreign Language requirements by studying Sanskrit, as part of the "SAFL: Samskritam as a Foreign Language" program coordinated by Samskrita Bharati. In Australia, the Sydney private boys' high school Sydney Grammar School offers Sanskrit from years 7 through to 12, including for the Higher School Certificate.
Index / Score: (11276, 0.8048174381256104)
Context: Elementary and secondary public schools are overseen by the North Carolina Department of Public Instruction. The North Carolina Superintendent of Public Instruction is the secretary of the North Carolina State Board of Education, but the board, rather than the superintendent, holds most of the legal authority for making public education policy. In 2009, the board's c